Este programa carga tu modelo VGG final y realiza clasificación de dígitos en tiempo real usando la cámara. Cada cuadro capturado se preprocesa exactamente igual que tus imágenes del dataset (recorte, escala de grises, resize, binarización OTSU e inversión). Después, el modelo predice el dígito y su probabilidad. La ventana principal muestra la cámara con el ROI marcado y las probabilidades de cada clase; además, muestra a un lado la versión preprocesada que realmente entra al modelo. El sistema actualiza las predicciones continuamente hasta que el usuario presiona q para salir.

In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

IMG_SIZE = 280
MODEL_PATH = "vgg_final_model_2.keras"

model = load_model(MODEL_PATH)
print("Modelo cargado ✔")

def preprocess_realtime(frame):
    """
    Preprocesa un frame EXACTAMENTE igual que el dataset:
    1. Recorte central (ROI)
    2. Gris
    3. Redimensionar a 280x280
    4. Binarización OTSU
    5. Invertir
    6. Expandir dims para que entre al modelo
    """

    h, w, _ = frame.shape

    # ---- 1. DEFINIR ROI CENTRAL ----
    side = min(h, w)
    cx, cy = w // 2, h // 2

    x1 = cx - side // 4
    x2 = cx + side // 4
    y1 = cy - side // 4
    y2 = cy + side // 4

    roi = frame[y1:y2, x1:x2]

    # ---- 2. GRAYSCALE ----
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

    # ---- 3. RESIZE ----
    resized = cv2.resize(gray, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)

    # ---- 4. BINARIZACIÓN OTSU ----
    _, thresh = cv2.threshold(resized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # ---- 5. INVERTIR ----
    inverted = cv2.bitwise_not(thresh)

    # ---- 6. NORMALIZAR + EXPANDIR ----
    norm = inverted.astype("float32") / 255.0
    model_input = norm.reshape(1, IMG_SIZE, IMG_SIZE, 1)

    # regresamos también la imagen preprocesada para visualizar
    return model_input, inverted, (x1, y1, x2, y2)

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("No se pudo abrir cámara.")
    exit()

print("Coloca el dígito dentro del cuadro amarillo. Presiona 'q' para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # --- PREPROCESAMIENTO ---
    model_input, processed_img, (x1, y1, x2, y2) = preprocess_realtime(frame)

    # --- PREDICCIÓN ---
    preds = model.predict(model_input, verbose=0)[0]
    class_id = np.argmax(preds)
    confidence = preds[class_id]

    # --- DIBUJAR ROI ---
    display = frame.copy()
    cv2.rectangle(display, (x1, y1), (x2, y2), (0,255,255), 2)

    # --- MOSTRAR PROBABILIDADES ---
    for i, p in enumerate(preds):
        color = (0,255,0) if i == class_id else (0,0,255)
        cv2.putText(display, f"{i}: {p:.3f}", (10, 30 + i*25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # --- MOSTRAR IMAGEN PROCESADA ---
    preview = cv2.cvtColor(processed_img, cv2.COLOR_GRAY2BGR)
    preview = cv2.resize(preview, (200, display.shape[0]))
    combined = np.hstack([display, preview])

    cv2.putText(combined, f"Pred: {class_id} ({confidence:.2f})",
                (10, combined.shape[0]-15),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0,255,0), 2)

    cv2.imshow("Clasificación tiempo real", combined)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Este programa implementa un sistema de OCR de múltiples dígitos en tiempo real usando tu modelo VGG final. La cámara captura cada cuadro y se procesa siguiendo exactamente el pipeline del profesor: conversión a escala de grises, desenfoque para reducir ruido, binarización dinámica con OTSU, inversión y dilatación para unir trazos. Luego se detectan contornos en la imagen dilatada y, para cada contorno válido, se extrae el recorte correspondiente y se preprocesa con el mismo flujo usado en tu dataset (resize, OTSU, invertir, normalizar). El modelo clasifica cada dígito encontrado y la predicción se dibuja sobre un bounding box en la imagen original. Se muestran dos ventanas: la imagen original con detecciones y la versión binarizada/dilatada usada para localizar los dígitos. El sistema corre de manera continua hasta que el usuario presiona Q para salir.

In [15]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

MODEL_PATH = "vgg_final_model_2.keras"
IMG_SIZE = 280

print("Cargando modelo...")
model = load_model(MODEL_PATH)
print("✔ Modelo cargado correctamente.\n")

def preprocess_for_model(roi_gray):
    """
    roi_gray: recorte en escala de grises del dígito
    """
    # 1) Resize a 280x280
    resized = cv2.resize(roi_gray, (IMG_SIZE, IMG_SIZE), interpolation=cv2.INTER_AREA)

    # 2) Binarización OTSU
    _, thresh = cv2.threshold(resized, 0, 255,
                              cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 3) Invertir (dígito blanco, fondo negro)
    inverted = cv2.bitwise_not(thresh)

    # 4) Normalizar + reshape
    norm = inverted.astype("float32") / 255.0
    model_input = norm.reshape(1, IMG_SIZE, IMG_SIZE, 1)

    return model_input


# ============================
# OCR EN TIEMPO REAL (PIPELINE DEL PROFE)
# ============================

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ No se pudo abrir la cámara")
    raise SystemExit

print("🎥 OCR multi-dígitos en tiempo real")
print("Presiona Q para salir.\n")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Frame no leído.")
        break

    # Copia para dibujar
    display = frame.copy()

    # ------------------------------
    # 1. Lee la imagen  -> ya la tenemos (frame)
    # 2. Convierte a escala de grises
    # ------------------------------
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # ------------------------------
    # 3. Aplica blur para reducir ruido
    #    (kernel se puede ajustar experimentalmente)
    # ------------------------------
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # ------------------------------
    # 4. Binariza (umbral dinámico OTSU) + invertimos
    #    para que el dígito sea blanco
    # ------------------------------
    _, binary = cv2.threshold(
        blur, 0, 255,
        cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU
    )

    # ------------------------------
    # 5. Dilatación para unir trazos
    #    (kernel e iteraciones ajustables)
    # ------------------------------
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(binary, kernel, iterations=2)

    # ------------------------------
    # 6. Buscar contornos en la imagen dilatada
    # ------------------------------
    contours, _ = cv2.findContours(
        dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
    )

    # Opcional: ordenar contornos de izquierda a derecha
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

    detected_digits = []

    # ------------------------------
    # 7. Iterar por cada contorno
    # ------------------------------
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)

        # Ignorar ruido muy pequeño
        if w < 20 or h < 20:
            continue

        # Genera bounding box
        cv2.rectangle(display, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Extrae pedazo de imagen de la IMAGEN ORIGINAL EN GRIS
        roi_gray = gray[y:y + h, x:x + w]

        # Preprocesar ROI para el modelo (tu pipeline)
        model_input = preprocess_for_model(roi_gray)

        # Predicción con el modelo
        preds = model.predict(model_input, verbose=0)[0]
        digit = np.argmax(preds)
        conf = preds[digit]

        detected_digits.append((digit, conf, x, y, w, h))

        # Dibujar etiqueta sobre el bounding box
        cv2.putText(
            display,
            f"{digit} ({conf:.2f})",
            (x, y - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.8,
            (0, 255, 0),
            2
        )

    # ------------------------------
    # 8. Visualización
    #    - display: imagen original + bounding boxes + etiquetas
    #    - dilated: "filtro" blur + binarización + dilatación
    # ------------------------------
    cv2.imshow("OCR - Original con detecciones", display)
    cv2.imshow("OCR - Blur + Binarizacion + Dilatacion", dilated)

    # Salida con 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Cargando modelo...
✔ Modelo cargado correctamente.

🎥 OCR multi-dígitos en tiempo real
Presiona Q para salir.

